# TransitKit v3.0 - Universal Exoplanet Analysis

## The Magic: ANY Planet, ANY Identifier, ONE Toolkit

This notebook demonstrates TransitKit's ability to handle **literally any exoplanet target** you can think of.

In [ ]:
from transitkit import UniversalTarget, quick_look
import warnings
warnings.filterwarnings('ignore')

## 1. The Universal Promise: Any Identifier Works

Watch how TransitKit handles completely different identifier formats:

In [ ]:
# All of these work with the EXACT same interface:

identifiers = [
    "WASP-39 b",           # Named planet (famous JWST target)
    "TIC 307210830",       # TESS Input Catalog ID
    "TOI-700 d",           # TESS Object of Interest (Earth-sized in HZ!)
    "Kepler-442b",         # Kepler planet
    "KIC 11904151",        # Kepler Input Catalog (Kepler-10)
    "KOI-7016.01",         # Kepler Object of Interest (candidate)
    "EPIC 201912552",      # K2 target (TRAPPIST-1!)
    "HD 209458 b",         # Henry Draper catalog
    "GJ 1214 b",           # Gliese catalog (super-Earth)
    "TRAPPIST-1 e",        # Famous system by name
    "Proxima Centauri b",  # Nearest exoplanet!
    "55 Cancri e",         # Ultra-short period
]

print("TransitKit v3.0 Universal Resolution Demo")
print("="*50)

for ident in identifiers[:5]:  # Demo first 5
    print(f"\n>>> UniversalTarget('{ident}')")
    try:
        target = UniversalTarget(ident, verbose=False)
        print(f"    ✅ Resolved: {target.ids.planet_name or target.ids.host_name}")
        print(f"    TIC: {target.tic}, Planets: {len(target.planets)}")
        if target.available_data.tess_sectors:
            print(f"    TESS sectors: {target.available_data.tess_sectors}")
    except Exception as e:
        print(f"    ⚠️ {e}")

## 2. Deep Dive: A Famous Target (WASP-39 b)

Let's see everything TransitKit knows about the famous JWST target:

In [ ]:
# Full resolution with verbose output
wasp39 = UniversalTarget("WASP-39 b", verbose=True)

In [ ]:
# Access all the cross-matched IDs
print("All Known Identifiers:")
print(f"  Planet name: {wasp39.ids.planet_name}")
print(f"  Host name: {wasp39.ids.host_name}")
print(f"  TIC: {wasp39.ids.tic}")
print(f"  Gaia DR3: {wasp39.ids.gaia_dr3}")
print(f"  2MASS: {wasp39.ids.two_mass}")
print(f"  Aliases: {wasp39.ids.aliases[:5]}...")  # First 5

In [ ]:
# Stellar parameters
star = wasp39.stellar
print("Host Star Parameters:")
print(f"  Coordinates: {star.ra:.6f}, {star.dec:.6f}")
print(f"  Teff: {star.teff} K")
print(f"  log(g): {star.logg}")
print(f"  [Fe/H]: {star.feh}")
print(f"  Radius: {star.radius} R☉")
print(f"  Mass: {star.mass} M☉")
print(f"  Distance: {star.distance} pc")

In [ ]:
# Planet parameters
for planet in wasp39.planets:
    print(f"\nPlanet: {planet.name}")
    print(f"  Period: {planet.period} days")
    print(f"  Radius: {planet.radius} R⊕")
    print(f"  Mass: {planet.mass} M⊕")
    print(f"  T_eq: {planet.teq} K")
    print(f"  Discovery: {planet.discovery_year} ({planet.discovery_facility})")

## 3. Multi-Mission Data: TESS + Kepler + K2

TransitKit automatically finds and downloads data from ALL missions:

In [ ]:
from transitkit import MultiMissionDownloader

# Pick a target with multi-mission coverage
target = UniversalTarget("Kepler-10 b", verbose=False)

print(f"Available data for {target.ids.planet_name}:")
print(f"  TESS sectors: {target.available_data.tess_sectors}")
print(f"  Kepler quarters: {target.available_data.kepler_quarters}")

In [ ]:
# Download everything (commented out to avoid long download)
# downloader = MultiMissionDownloader(target)
# data = downloader.download_all()
# 
# print(data.summary())
# 
# # Stitch together for 10+ year baseline!
# combined = data.stitch()
# print(f"\nTotal baseline: {combined.total_timespan:.0f} days ({combined.total_timespan/365:.1f} years)")

## 4. JWST Spectroscopy: Atmospheric Characterization

In [ ]:
from transitkit import JWSTSpectroscopy

# WASP-39 b has JWST data!
target = UniversalTarget("WASP-39 b", verbose=False)

print("JWST Data Available:")
print(f"  Programs: {target.available_data.jwst_programs}")
print(f"  Instruments: {target.available_data.jwst_instruments}")
print(f"  Has transmission spectrum: {target.available_data.has_transmission_spectrum}")

In [ ]:
# Get and analyze spectrum (commented out - requires download)
# jwst = JWSTSpectroscopy(target)
# spectrum = jwst.get_transmission_spectrum()
# 
# # Detect molecules
# molecules = jwst.detect_molecules(spectrum)
# 
# print("\nMolecule Detection Results:")
# for m in molecules:
#     if m.detected:
#         print(f"  ✅ {m.molecule}: {m.significance:.1f}σ at {m.wavelength_range[0]:.2f}-{m.wavelength_range[1]:.2f} μm")

## 5. ML Transit Detection: Find New Planets

In [ ]:
from transitkit import MLTransitDetector, detect_transits
import numpy as np

# Simulate a light curve with a transit
np.random.seed(42)
time = np.linspace(0, 27, 10000)  # 27 days of data
flux = np.ones_like(time) + np.random.normal(0, 0.001, len(time))

# Inject a transit: P=3.5 days, depth=1000 ppm
period = 3.5
t0 = 1.2
duration = 0.1  # days
depth = 0.001  # 1000 ppm

for i in range(int(27/period) + 1):
    transit_time = t0 + i * period
    mask = np.abs(time - transit_time) < duration/2
    flux[mask] -= depth

# Detect it!
candidates = detect_transits(time, flux, min_snr=5)

print("Transit Detection Results:")
print(f"  Injected: P={period} d, depth={depth*1e6:.0f} ppm")
print(f"  Found: {len(candidates)} candidate(s)")

if candidates:
    c = candidates[0]
    print(f"\n  Best candidate:")
    print(f"    Period: {c.period:.4f} d (error: {abs(c.period-period)/period*100:.1f}%)")
    print(f"    Depth: {c.depth:.0f} ppm")
    print(f"    SNR: {c.snr:.1f}")
    print(f"    ML score: {c.ml_score:.2f}")
    print(f"    FP probability: {c.fp_probability:.0%}")

## 6. Full Analysis Pipeline: One Command

In [ ]:
# The ultimate one-liner (commented out - takes time)
# 
# target = UniversalTarget("TOI-700 d")
# results = target.analyze()  # Downloads, detects, fits, everything!
# target.export("toi700d_paper/")  # Publication-ready output

## 7. Batch Processing: Analyze Many Targets

In [ ]:
# Process a list of targets
targets_to_analyze = [
    "TOI-700 d",      # Earth-sized in HZ
    "LHS 1140 b",     # Rocky super-Earth
    "K2-18 b",        # Potential water world
    "TRAPPIST-1 e",   # TRAPPIST system
]

print("Batch Resolution Demo:")
print("="*60)

for name in targets_to_analyze:
    try:
        t = UniversalTarget(name, verbose=False)
        planet = t.planets[0] if t.planets else None
        
        print(f"\n{name}:")
        print(f"  TIC: {t.tic}")
        if planet:
            print(f"  Radius: {planet.radius:.2f} R⊕" if planet.radius else "  Radius: --")
            print(f"  T_eq: {planet.teq:.0f} K" if planet.teq else "  T_eq: --")
        print(f"  TESS sectors: {len(t.available_data.tess_sectors)}")
        print(f"  JWST programs: {len(t.available_data.jwst_programs)}")
    except Exception as e:
        print(f"\n{name}: ⚠️ {e}")

## 8. Export to JSON for External Tools

In [ ]:
target = UniversalTarget("WASP-39 b", verbose=False)

# Export all data as JSON
json_data = target.to_json()
print(json_data[:1000] + "...")

## Summary

TransitKit v3.0 provides a **truly universal** interface for exoplanet analysis:

| Feature | Before (v2.x) | Now (v3.0) |
|---------|---------------|------------|
| Target input | TIC ID only | ANY identifier |
| Missions | TESS only | TESS + Kepler + K2 + JWST |
| Resolution | Manual lookup | Automatic cross-match |
| Detection | BLS only | BLS + TLS + ML ensemble |
| Spectroscopy | None | JWST transmission + molecules |
| Output | Data only | Publication-ready |

**One toolkit. Any planet. Complete analysis.**